# ROI Figure Notebook

This notebook is used to create the figure displaying the parcellation used in these analyses.

In [1]:
import os
import os.path as op
from surfer import Brain
import matplotlib as plt
from nipy.labs import viz
import nibabel as nib
from IPython.display import display, Image
from matplotlib import *
import matplotlib.pyplot as plt
import sys
import pylab as pl
from scipy.ndimage import binary_erosion
import lyman
import moss
import numpy as np
import seaborn as sns
from mayavi import mlab
sns.set(context="paper", style="white")
%matplotlib inline

In [1]:
home_dir = '/home/iballard/mag/'

In [3]:
# views = dict(pfc=dict(lh=[135, 80, 290, [20, 15, -5]],
#                       rh=[45, 80, 290, [-20, 15, -5]]),
#              net=dict(lh=[230, 65, 335, [40, 15, -20]],
#                       rh=[-105, 105, 290, [0, -15, 0]]))
views = dict(pfc=dict(lh=[0, 90, 290, [40, 15, 20]],
                      rh=[0, 90, 290, [40, 15, 20]]))

In [4]:
opts = dict(width=800, height=800, background="white")
ctab = '/home/iballard/mag/data/fsaverage/label/pfc.ctab'

In [39]:
os.chdir('/home/iballard/mag/data/fsaverage/label')
rois = ['_amfg',
        '_pmfg',
        '_fpc',
        '_ifg',
        '_psfs',
        '_ifs',
       '_ains']
for hemi in ['lh','rh']:
    annot = hemi + '.pfc_rois.annot'
    if os.path.exists(annot):
        os.remove(annot)
        
    cmd = ['mris_label2annot','--s','fsaverage',
           '--ctab', ctab,
          '--h',hemi,'--a','pfc_rois','--nhits', 'nhits.mgh']
    for r in rois:
        cmd += ['--l',hemi + '.yeo17' + r + '.label']
        
    cmd += ['--l',hemi + '.vmpfc_bartre.label']
    
    print ' '.join(cmd)
    os.system(' '.join(cmd))


mris_label2annot --s fsaverage --ctab /home/iballard/mag/data/fsaverage/label/pfc.ctab --h lh --a pfc_rois --nhits nhits.mgh --l lh.yeo17_amfg.label --l lh.yeo17_pmfg.label --l lh.yeo17_fpc.label --l lh.yeo17_ifg.label --l lh.yeo17_psfs.label --l lh.yeo17_ifs.label --l lh.yeo17_ains.label --l lh.vmpfc_bartre.label
mris_label2annot --s fsaverage --ctab /home/iballard/mag/data/fsaverage/label/pfc.ctab --h rh --a pfc_rois --nhits nhits.mgh --l rh.yeo17_amfg.label --l rh.yeo17_pmfg.label --l rh.yeo17_fpc.label --l rh.yeo17_ifg.label --l rh.yeo17_psfs.label --l rh.yeo17_ifs.label --l rh.yeo17_ains.label --l rh.vmpfc_bartre.label


Use PySurfer to draw the surfaces with the full annotation

In [29]:
annot_shots = dict(pfc=dict(), net=dict())
for hemi in ["lh", "rh"]:
    b = Brain("fsaverage", hemi, "semi7", config_opts=opts)
    b.add_annotation("pfc_rois", alpha=.8, borders=False)
    for view in ["pfc"]:
        mlab.view(*views[view][hemi])
        annot_shots[view][hemi] = mlab.screenshot()
        mlab.savefig(home_dir + '/figs/' + view + '_' + hemi + '_rois.png')
    b.close()

In [3]:
%gui qt
hemi = "lh"
surf = "semi7"
brain = Brain("fsaverage", hemi, surf)


brain.add_label("vmpfc_bartre")


In [2]:
import os
from surfer import Brain, project_volume_data
%gui qt

brain = Brain("fsaverage", "lh", "inflated")

"""
Get a path to the volume file.
"""
volume_file = home_dir + "/rois/vmpfc_bartre_5way.nii.gz"

"""
There are two options for specifying the registration between the volume and
the surface you want to plot on. The first is to give a path to a
Freesurfer-style linear transformation matrix that will align the statistical
volume with the Freesurfer anatomy.

Most of the time you will be plotting data that are in MNI152 space on the
fsaverage brain. For this case, Freesurfer actually ships a registration matrix
file to align your data with the surface.
"""
reg_file = os.path.join(os.environ["FREESURFER_HOME"],
                        "average/mni152.register.dat")
zstat = project_volume_data(volume_file, "lh", reg_file)

"""
Note that the contours of the fsaverage surface don't perfectly match the
MNI brain, so this will only approximate the location of your activation
(although it generally does a pretty good job). A more accurate way to
visualize data would be to run the MNI152 brain through the recon-all pipeline.

Alternatively, if your data are already in register with the Freesurfer
anatomy, you can provide project_volume_data with the subject ID, avoiding the
need to specify a registration file.

By default, 3mm of smoothing is applied on the surface to clean up the overlay
a bit, although the extent of smoothing can be controlled.
"""
zstat = project_volume_data(volume_file, "lh",
                            subject_id="fsaverage", smooth_fwhm=0.5)

"""
Once you have the statistical data loaded into Python, you can simply pass it
to the `add_overlay` method of the Brain object.
"""
brain.add_overlay(zstat, min=.1, max=2)


brain.show_view("medial")

mri_vol2surf --mov /home/iballard/mag//rois/vmpfc_bartre_5way.nii.gz --hemi lh --surf white --reg /software/freesurfer/v5.3.0/average/mni152.register.dat --projfrac-avg 0 1 0.1 --surf-fwhm 3 --o /tmp/pysurfer-v2sVsY8iJ.mgz
mri_vol2surf --mov /home/iballard/mag//rois/vmpfc_bartre_5way.nii.gz --hemi lh --surf white --regheader fsaverage --projfrac-avg 0 1 0.1 --surf-fwhm 0.5 --o /tmp/pysurfer-v2sWHpNsM.mgz


/home/iballard/anaconda/lib/python2.7/site-packages/mayavi/tools/camera.py:288: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if focalpoint is not None and not focalpoint == 'auto':


((0.0, 90.0, 432.11123657226562, array([ 0.,  0.,  0.])), -90.0)